Exploratory Data Analysis 🏗️📊
Configuration

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import dash
import dash_table
from dash import dcc, html
import plotly.express as px
# Conecta ao banco de dados SQLite
conn = sqlite3.connect('C:/Users/santo/Documents/GitHubPublished/DataScienceProject/database/ecommerceProject.db')

# Dicionário de queries SQL
dictQuery = {
    "queryCustomers": 'SELECT * FROM customers LIMIT 5',
    "queryOrders": 'SELECT * FROM orders LIMIT 5',
    "queryProducts": 'SELECT * FROM products LIMIT 5',
    "queryCategories": 'SELECT * FROM categories LIMIT 5',
    "queryOrderItens": 'SELECT * FROM order_items LIMIT 5',
    "queryCountProducts": 'SELECT COUNT(*) AS total_products FROM products',
    "queryCountCategories": 'SELECT COUNT(*) AS total_categories FROM categories',
    "queryAvgSales": 'SELECT AVG(order_value) AS avg_sales FROM orders',
    "queryDistinctCategories": 'SELECT DISTINCT category_name FROM categories',
    "queryProductsByCategories": '''
                SELECT c.category_name, COUNT(DISTINCT p.product_name) AS count
                FROM categories c
                JOIN products p
                ON c.category_id = p.category_id
                GROUP BY c.category_name
                ORDER BY c.category_name''',
    "queryTop10CustomerValue":'''
                SELECT
                c.customer_id,
                (SUM(o.order_value)) AS total
                FROM customers c
                JOIN orders o
                ON c.customer_id = o.customer_id
                GROUP BY c.customer_id
                ORDER BY total DESC
                LIMIT 10''',
}

def runQuery(queryKey, params=()):
    """Executa uma consulta SQL e retorna o resultado como um DataFrame.

    Args:
        queryKey (str): Chave da consulta no dicionário `dictQuery`.
        params (tuple, optional): Parâmetros para a consulta SQL. Padrão é ().

    Returns:
        pandas.DataFrame: DataFrame com o resultado da consulta.
    """
    query = dictQuery[queryKey]  # Obtém a consulta SQL do dicionário
    return pd.read_sql(query, conn, params=params)

C:\Users\santo\AppData\Local\Temp\ipykernel_6640\2609512629.py:5: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


Exploratory Data Analysis 🏗️📊
Queries

In [2]:
# Executa as consultas para obter os DataFrames
tCustomers = runQuery("queryCustomers")
tOrders = runQuery("queryOrders")
tProducts = runQuery("queryProducts")
tCategories = runQuery("queryCategories")
tOrderItens = runQuery("queryOrderItens")
tCountProducts = runQuery("queryCountProducts")
tCountCategories = runQuery("queryCountCategories")
tAvgSales = runQuery("queryAvgSales")
tDistinctCategories = runQuery("queryDistinctCategories")
tProductsByCategories = runQuery("queryProductsByCategories")
tTop10CustomerValue = runQuery("queryTop10CustomerValue")

# Inicia o aplicativo Dash
app = dash.Dash(__name__)

# Define o layout com tabelas interativas
app.layout = html.Div([
    html.H1("Tabelas do Banco de Dados E-commerce (Consultas Básicas)", style={"textAlign": "center"}),
    
    html.H2("Clientes"),
    dash_table.DataTable(
        id='customers-table',
        columns=[{"name": col, "id": col} for col in tCustomers.columns],
        data=tCustomers.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Pedidos"),
    dash_table.DataTable(
        id='orders-table',
        columns=[{"name": col, "id": col} for col in tOrders.columns],
        data=tOrders.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Produtos"),
    dash_table.DataTable(
        id='products-table',
        columns=[{"name": col, "id": col} for col in tProducts.columns],
        data=tProducts.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Categorias"),
    dash_table.DataTable(
        id='categories-table',
        columns=[{"name": col, "id": col} for col in tCategories.columns],
        data=tCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Itens de Pedido"),
    dash_table.DataTable(
        id='order-itens-table',
        columns=[{"name": col, "id": col} for col in tOrderItens.columns],
        data=tOrderItens.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Contagem de Produtos"),
    dash_table.DataTable(
        id='count-products-table',
        columns=[{"name": col, "id": col} for col in tCountProducts.columns],
        data=tCountProducts.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Contagem de Categorias"),
    dash_table.DataTable(
        id='count-categories-table',
        columns=[{"name": col, "id": col} for col in tCountCategories.columns],
        data=tCountCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Média de Vendas"),
    dash_table.DataTable(
        id='avg-sales-table',
        columns=[{"name": col, "id": col} for col in tAvgSales.columns],
        data=tAvgSales.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Categorias Distintas"),
    dash_table.DataTable(
        id='distinct-categories-table',
        columns=[{"name": col, "id": col} for col in tDistinctCategories.columns],
        data=tDistinctCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),

    html.H1("Tabelas do Banco de Dados E-commerce (Agregações, Agrupamentos e Joins)", style={"textAlign": "center"}),
    html.H2("Produtos por Categorias"),
    dash_table.DataTable(
        id='products-by-categories-table',
        columns=[{"name": col, "id": col} for col in tProductsByCategories.columns],
        data=tProductsByCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Top 10 Clientes por Valor de Compra"),
    dash_table.DataTable(
        id='top10-customer-value-table',
        columns=[{"name": col, "id": col} for col in tTop10CustomerValue.columns],
        data=tTop10CustomerValue.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
])

if __name__ == '__main__':
    app.run(debug=True)  

In [3]:
conn.close()  # Fecha a conexão com o banco de dados